In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Solving a Quadratic Equation with  Z3

In this notebook, we solve the following puzzle:

<blockquote id="fancy">
   A train travels at a uniform speed for 360 miles.  
   The train would have taken 48 minutes less to travel the same distance 
   if it had been 5 miles per hour faster. <br/>
    
   **Find the speed of the train**.
</blockquote>

In [2]:
import z3

If $v$ is the velocity of the train (measured in miles per hour), then the time $t$ in hours to cover a distance of 360 miles is determined by the formula
$$ 360 = t \cdot v. $$
If the train is 5 miles per hour faster, then the time to travel the same idstance is 48 minutes less.  
As $\displaystyle 48 = \frac{4}{5} \cdot 60$, this is equivalent to the equation
$$ 360 = \left(t - \frac{4}{5}\right) \cdot (v + 5). $$
Effectively, we therefore have to solve the following *non-linear* system of equations:
$$ \left\{
   \begin{array}{lcl}
     360 & = & t \cdot v \\
     360 & = & \left(t - \frac{4}{5}\right) \cdot (v + 5)
   \end{array}
   \right\}
$$
If we solve the first equation for $t$ and substitute the resulting value for $t$ into the second equation, 
we can transform this system of equations into a single quadratic equation.  However, that gets tedious fast. 
Instead, we will use `Z3` to solve this system of equations.

We start by declaring $t$ and $v$ as variables ranging over the set of real numbers $\mathbb{R}$.

In [3]:
t = z3.Real('t')
v = z3.Real('v')

Next, we create a *solver* object.

In [4]:
S = z3.Solver()

We can add the constraints to this solver object.  The fraction $\displaystyle \frac{4}{5}$ is created via the function `Q`. 

In [5]:
S.add(360 == t * v)
S.add(360 == (t - z3.Q(4, 5)) * (v + 5))

Actually, this system of equations given so far has two solutions, one of which is negative.
In order to exclude the negative solution, we have to add the constraints that the time $t$ has to be positive.

In [6]:
S.add(t > 0)

Let us test whether there is a solution.

In [7]:
S.check()

sat

All right, our system of equation is solvable. 

We extract the solution from the solver.

In [8]:
solution = S.model()
solution

[t = 8, v = 45, /0 = [(360, 8) -> 45, else -> 0]]

We convert the speed into a string using the method `as_decimal`.
Here, the number `17` is the *precision*, i.e. the number of digits following the decimal point.  Since the method `as_decimal` returns a string, we have to convert this string into a floating point number.

In [9]:
velocity = float(solution[v].as_decimal(17))
velocity

45.0

In [10]:
print(f'The train was driving with a velocity of {velocity} miles per hour.')

The train was driving with a velocity of 45.0 miles per hour.


Let us check that this is correct.

In [11]:
time = float(solution[t].as_decimal(17))

In [12]:
time * velocity

360.0

In [13]:
(time - 4/5) * (velocity + 5)

360.0